### Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from masserstein import NMRSpectrum, estimate_proportions
import matplotlib.pyplot as plt
import pulp
import pickle

### Data

In [ ]:
experiment_name = 'PMG 284 monitoring with full mixture spectrum'

In [ ]:
if experiment_name == 'PMG 284 monitoring':
    whole_mixture_or_regions = 'regions'
elif experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    whole_mixture_or_regions = 'whole'

if whole_mixture_or_regions == 'regions':
    data_path = '../../data/PMG_284/'
    mixture_path = data_path + '284_edited_corrected.csv'

elif whole_mixture_or_regions == 'whole':
    data_path = '../../data/PMG_284_full_mixture_spectrum/'
    mixture_path = data_path + '284_edited_corrected_no_cut.csv'

mixture_separator = '\t'

#### Mixture in time

In [ ]:
mixture_time_data = pd.read_csv(mixture_path, sep = mixture_separator).iloc[:,:-2]

In [ ]:
mixture_time_data

In [ ]:
names = ['ppm'] + ['t' + str(nb) for nb in range(1, mixture_time_data.shape[1])]

In [ ]:
mixture_time_data.columns = names

In [ ]:
def load_spectrum(mixture_time_data, moment_of_time):
    ppm = mixture_time_data['ppm']
    intensity = mixture_time_data['t'+str(moment_of_time)]
    sp = NMRSpectrum(confs = list(zip(ppm, intensity)))
    return sp

In [ ]:
#%matplotlib notebook

In [ ]:
plt.gca().invert_xaxis()
NMRSpectrum.plot_all([load_spectrum(mixture_time_data, 1),
                     load_spectrum(mixture_time_data, 999)], profile=True)

#### Reagents

In [ ]:
reagents_sp = []

mix920 = pd.DataFrame(load_spectrum(mixture_time_data, 920).confs)

#2-pentene
reagent0 = mix920[mix920[0].apply(lambda x: (x>5.3 and x<5.6)
                                 )]
ppm = reagent0.iloc[:,0]
ints = reagent0.iloc[:,1]
reagents_sp.append(NMRSpectrum(confs = list(zip(ppm, ints))))


#triethylsilane                           
reagent1 = mix920[mix920[0].apply(lambda x: (x>3.6 and x<3.95)
                                 )]
ppm = reagent1.iloc[:,0]
ints = reagent1.iloc[:,1]
reagents_sp.append(NMRSpectrum(confs = list(zip(ppm, ints))))

#product                              
reagent2 = mix920[mix920[0].apply(lambda x: (x>1.16 and x<1.36) 
                                 )]
ppm = reagent2.iloc[:,0]
ints = reagent2.iloc[:,1]
reagents_sp.append(NMRSpectrum(confs = list(zip(ppm, ints))))

In [ ]:
mix920_sp = load_spectrum(mixture_time_data, 920)
plt.gca().invert_xaxis()
#plt.xlim(1.8, 1.14)
NMRSpectrum.plot_all([mix920_sp] + reagents_sp, profile=True)

### Mixture

In [ ]:
if whole_mixture_or_regions == 'whole':
    pass
elif whole_mixture_or_regions == 'regions':
    mixture_time_data = mixture_time_data[mixture_time_data['ppm'].apply(lambda x: 
                                                        (x>5.3 and x<5.6) or 
                                                        (x>3.6 and x<3.95) or 
                                                        (x>1.16 and x<1.36) 
                                     )]

In [ ]:
plt.gca().invert_xaxis()
NMRSpectrum.plot_all([load_spectrum(mixture_time_data, 1),
                 load_spectrum(mixture_time_data, 999)], profile=True)

### Saving preprocessed spectra

In [ ]:
np.savetxt(data_path + 'preprocessed_pentene.csv', reagent0, delimiter = '\t')
np.savetxt(data_path + 'preprocessed_triethylsilane.csv', reagent1, delimiter = '\t')
np.savetxt(data_path + 'preprocessed_product.csv', reagent2, delimiter = '\t')

if whole_mixture_or_regions == 'whole':
    np.savetxt(data_path + 'preprocessed_mixture_whole.csv', mixture_time_data, delimiter = '\t')
    
elif whole_mixture_or_regions == 'regions':
    np.savetxt(data_path + 'preprocessed_mixture_cut_to_regions.csv', mixture_time_data, delimiter = '\t')